In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import load_boston
import pandas as pd

In [149]:

df = pd.read_csv(load_boston()["filename"], skiprows=1)
df.head()
X = df.iloc[:,0:12]
y = df.loc[:,['MEDV']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42 )  
X_train["MEDV"]=y["MEDV"]


# Y=y["MEDV"]
# print(np.mean(Y))

# for x in X_train.iloc[:,:-1]:
# #     print(X[x].head())
#     for z in X[x]:
#         a = X[X[x]>z]
#         print(a.head())
#         break

# X_train.iloc[:,-1:]
# for x in X_train:
#     print(X_train[x])



<ipython-input-149-aec92ccd1a3a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["MEDV"]=y["MEDV"]


[         CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
 5     0.02985   0.0   2.18     0  0.458  6.430  58.7  6.0622    3  222   
 116   0.13158   0.0  10.01     0  0.547  6.176  72.5  2.7301    6  432   
 45    0.17142   0.0   6.91     0  0.448  5.682  33.8  5.1004    3  233   
 16    1.05393   0.0   8.14     0  0.538  5.935  29.3  4.4986    4  307   
 468  15.57570   0.0  18.10     0  0.580  5.926  71.0  2.9084   24  666   
 ..        ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
 106   0.17120   0.0   8.56     0  0.520  5.836  91.9  2.2110    5  384   
 270   0.29916  20.0   6.96     0  0.464  5.856  42.1  4.4290    3  223   
 348   0.01501  80.0   2.01     0  0.435  6.635  29.7  8.3440    4  280   
 435  11.16040   0.0  18.10     0  0.740  6.629  94.6  2.1247   24  666   
 102   0.22876   0.0   8.56     0  0.520  6.405  85.4  2.7147    5  384   
 
      PTRATIO       B  MEDV  
 5       18.7  394.12  28.7  
 116     17.8  393.30  21.2  
 45     

In [102]:
def H(R):
    Y = np.array(R)
    m = np.mean(Y)
    res = 0
    for el in R.values:
        res += (el - m)**2
    return res/len(R)
        

In [120]:
def ig(R, Rl,Rr):
    return H(R.iloc[:,-1:])-(H(Rl.iloc[:,-1:]) + H(Rr.iloc[:,-1:]))

In [160]:
def spliting(X):
    parts = [[],[]]
    InG = - 10*8
    for param in X:
        for val in X[param]:
            X1 = X[X[param]>=val]
            X2 = X[X[param]< val]
            if len(X1) == 0 or len(X2) == 0:
                continue
            InG_tmp =Ig(X, X1, X2)
            if InG < InG_tmp:
                InG == InG_tmp
                parts[0] = X1
                parts[1] = X2
    return parts, InG     
        

In [161]:
def buildTree(X, deep=3, elem_in_leaf=3, igdelta= 10*-5):
    print(deep)
    deep-=1
    tree=[]

    if not isinstance(X, list):
        X=[{"data":X,"Ig":1, "frozen_node":False}]
        
    not_blocked_nodes = [el["frozen_node"] for el in X]
    if deep == 0 or sum(not_blocked_nodes) == 0:
        return X
    
    for part in X:
        if part["frozen_node"]:
            tree.append(part)
        else:
            split_res,new_ing = spliting(part["data"])
            if (new_ing - part["Ig"] < igdelta):
                part["frozen_node"] = True
                tree.append(part)
            else:
                tree.append([{"data":split_res[0],"Ig":ing},{"data":split_res[1],"Ig":ing}])
    return buildTree(tree, deep=deep, elem_in_leaf=elem_in_leaf, imptov=imptov)
        
#     tree = []
#     InG = 0
#     iteration = 0
#     tree,InG = spliting(X)
#     while iteration < deep
    
    

In [162]:
buildTree(X)

3
[        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273   
502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273   
503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273   
504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273   
505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273   

     PTRATIO       B  
0       15.3  396.90  
1       17.8  396.90  
2       17.8  392.83  
3       18.7  394.63  
4    

[        CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD  TAX  \
 0    0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296   
 1    0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242   
 2    0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242   
 3    0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222   
 4    0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222   
 ..       ...   ...    ...   ...    ...    ...   ...     ...  ...  ...   
 501  0.06263   0.0  11.93     0  0.573  6.593  69.1  2.4786    1  273   
 502  0.04527   0.0  11.93     0  0.573  6.120  76.7  2.2875    1  273   
 503  0.06076   0.0  11.93     0  0.573  6.976  91.0  2.1675    1  273   
 504  0.10959   0.0  11.93     0  0.573  6.794  89.3  2.3889    1  273   
 505  0.04741   0.0  11.93     0  0.573  6.030  80.8  2.5050    1  273   
 
      PTRATIO       B  
 0       15.3  396.90  
 1       17.8  396.90  
 2       17.8  392.83  
 3       18.7 

In [136]:
z=[]
a= [1,2,3]
b=[[1,2,3],[4,5,6]]
z.append(a)
z.extend(b)
z

[[1, 2, 3], [1, 2, 3], [4, 5, 6]]

In [163]:
z= [True, False, True]
sum(z)

2